# Finding Y-Bus, Z-Bus, V-Bus

In [ ]:
import numpy as np
z = [0,1,0,1.0,0,2,0,0.8,1,2,0,0.4,1,3,0,0.2,2,3,0,0.2,3,4,0,0.08]
arr = np.array(z)
zdata = arr.reshape(6,4)
nd=[]
for i in range(len(zdata)):
    if (zdata[i][0]!=0):
        nd.append(zdata[i])
zdata1=np.array(nd)
inl = zdata[:,0]
nl_tmp = inl.astype('int64')
nl = inl.astype('complex')
inr = zdata[:,1]
nr_tmp = inr.astype('int64')
nr = inr.astype('complex')
R = zdata[:,2]
X = zdata[:,3]
nbr = len(nl)
nbus = int(max(max(nl_tmp),max(nr_tmp)))
#Y = np.zeros((nbus,nbus))
Z = [a+1j*b for(a,b) in zip(R,X)]
#y = np.ones((nbr,1))/Z
y = []
for i in (Z):
    y.append((1/i))
y = np.array(y)
for i in range (0,len(nl)):
    nl[i]=nl[i]+0j
for j in nr:
    nr[i]=nr[i]+0j
    
Y=[]
for i in range(nbus*4):
    Y.append(0j)
Y = np.array(Y)
Y = Y.reshape(nbus,nbus)
a=[]
v=[]
#For diagonal 
for n in range(0,nbus):
    for m in range(0,nbr):
        if nl[m] == n+1 or nr[m] == n+1:
            Y[n,n]=Y[n,n] + y[m]
a.append([1,1])
v.append(Y[1,1])
a.append([0,0])
v.append(Y[0,0])
a.append([2,2])
v.append(Y[2,2])
a.append([3,3])
v.append(Y[3,3])

inl = zdata1[:,0]
nl_tmp = inl.astype('int64')
nl = inl.astype('complex')
inr = zdata1[:,1]
nr_tmp = inr.astype('int64')
nr = inr.astype('complex')
R = zdata1[:,2]
X = zdata1[:,3]
nbr = len(nl)
nbus = int(max(max(nl_tmp),max(nr_tmp)))
#Y = np.zeros((nbus,nbus))
Z = [a+1j*b for(a,b) in zip(R,X)]
#y = np.ones((nbr,1))/Z
y = []
for i in (Z):
    y.append((1/i))
d={}
for i in range (len(nl_tmp)):
    if(nl_tmp[i] not in d):
        d[int(nl_tmp[i])]=int(nr_tmp[i])
    else:
        if(type(d[nl_tmp[i]])=='list'):
            d[int(nl_tmp[i])].append(int(nr_tmp[i]))
        else:    
            l=[]
            l.append(int(d[nl_tmp[i]]))
            l.append(int(nr_tmp[i]))
            d[int(nl_tmp[i])]=l

for i in range(len(nl_tmp)):    
    for j in range(len(nl_tmp)):
        if ((nl_tmp[j] > 0) and (nr_tmp[j] > 0)):
                    if(nl_tmp[i] in d.keys()):
                        if(str(type(d[nl_tmp[i]]))=="<class 'list'>"):
                            if (int(nr_tmp[j]) in d[nl_tmp[i]]):
                                if(([i,j] in a)):
                                    val=v[a.index([i,j])]
                                    Y[i,j]=val
                                elif(([j,i] in a)):
                                    val=v[a.index([j,i])]
                                    Y[j,i]=val
                                else:
                                    for k in range(i,j):
                                        Y[i,j]=-y[k]
                                        Y[j,i]=Y[i,j]
                                        a.append([i,j])
                                        a.append([j,i])
                                        v.append(Y[i,j])
                                        v.append(Y[j,i])
                        else:
                            if(int(nr_tmp[j]) == int(d[nl_tmp[i]])):
                                if(([i,j] in a)):
                                    val=v[a.index([i,j])]
                                    Y[i,j]=val
                                elif(([k,i] in a)):
                                    val=v[a.index([j,i])]
                                    Y[j,i]=val
                                else:
                                    for k in range(i,j):
                                        Y[i,j]=-y[k]
                                        Y[j,i]=Y[i,j]
                                        a.append([i,j])
                                        a.append([j,i])
                                        v.append(Y[i,j])
                                        v.append(Y[j,i])

Y[nbr-2,nbr-1]=-y[nbr-1]
Y[nbr-1,nbr-2]=-y[nbr-1]

Zbus = np.linalg.inv(Y)
Ibus = [0-1j*1.1, 0-1j*1.25,0,0]
Vbus = Ibus*Zbus

vbus=[]
for i in Vbus:
    vbus.append(np.round(sum(i),3))
print('-----Ybus----- ')
print(Y)

print('-----Zbus-----')
print(Zbus)

print('-----Vbus-----')
for i in vbus:
    print(i)

-----Ybus----- 
[[ 0. -8.5j  -0. +2.5j  -0. +5.j    0. +0.j  ]
 [-0. +2.5j   0. -8.75j -0. +5.j    0. +0.j  ]
 [-0. +5.j   -0. +5.j    0.-22.5j  -0.+12.5j ]
 [ 0. +0.j    0. +0.j   -0.+12.5j   0.-12.5j ]]
-----Zbus-----
[[0.+0.5j   0.+0.4j   0.+0.45j  0.+0.45j ]
 [0.+0.4j   0.+0.48j  0.+0.44j  0.+0.44j ]
 [0.+0.45j  0.+0.44j  0.+0.545j 0.+0.545j]
 [0.+0.45j  0.+0.44j  0.+0.545j 0.+0.625j]]
-----Vbus-----
(1.05+0j)
(1.04+0j)
(1.045+0j)
(1.045+0j)


# Gaus Seidal Method

In [ ]:
dx = 1
x = 2
Iter = 0
print('Iter        g          dx              x')
while abs(dx) >= 0.001 and Iter < 100:
    Iter = Iter + 1
    g = (-1/9*x**3)+(6/9*x**2)+(4/9)
    dx = g-x
    x = x+dx
    print(Iter,'      ',round(g,4),'    ',round(dx,4),'        ',round(x,4))

Iter        g          dx              x
1        2.2222      0.2222          2.2222
2        2.5173      0.2951          2.5173
3        2.8966      0.3793          2.8966
4        3.3376      0.441          3.3376
5        3.7398      0.4022          3.7398
6        3.9568      0.217          3.9568
7        3.9988      0.042          3.9988
8        4.0      0.0012          4.0
9        4.0      0.0          4.0


# Newton Rapson Method

In [ ]:
dx = 1
x = int(input('Enter initial estimate: '))
Iter = 0
print('dc           j           dx           x')
while abs(dx) >= 0.001 and Iter < 100:
    Iter = Iter+1
    dc = 0 - ((x**3) - (6*x**2) + (9*x) - 4)
    j = (3*x**2) - (12*x) + 9
    dx = dc/j
    x = x+dx
    print(round(dc,4),'   ',round(j,4),'   ',round(dx,4),'   ',round(x,4))

Enter initial estimate: 6
dc           j           dx           x
-50     45     -1.1111     4.8889
-13.4431     22.037     -0.61     4.2789
-2.9981     12.5797     -0.2383     4.0405
-0.3748     9.4914     -0.0395     4.0011
-0.0095     9.0126     -0.0011     4.0
-0.0     9.0     -0.0     4.0
